In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

In [2]:
classes = ('Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

In [3]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

In [4]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [5]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=2)

Files already downloaded and verified


In [ ]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

AlexNet_model.classifier[4] = nn.Linear(4096,1024)
AlexNet_model.classifier[6] = nn.Linear(1024,10)
AlexNet_model.classifier[0] = nn.Dropout(p=0.7)
AlexNet_model.classifier[3] = nn.Dropout(p=0.7)

criterion = nn.CrossEntropyLoss()

#Optimizer(SGD)
optimizer = optim.SGD(AlexNet_model.parameters(), lr=0.001, momentum=0.9)

AlexNet_model.to('cuda:0')

for epoch in range(10): 
    lossval = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = AlexNet_model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        lossval += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, lossval / 2000))
            lossval = 0.0

print('Finished Training of AlexNet')
#Testing Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = AlexNet_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = AlexNet_model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

cuda:0
[1,  2000] loss: 1.607
[1,  4000] loss: 1.284
[1,  6000] loss: 1.167
[1,  8000] loss: 1.096
[1, 10000] loss: 1.042
[1, 12000] loss: 1.007
[2,  2000] loss: 0.914
[2,  4000] loss: 0.906
[2,  6000] loss: 0.895
[2,  8000] loss: 0.872
[2, 10000] loss: 0.867
[2, 12000] loss: 0.846
[3,  2000] loss: 0.787
[3,  4000] loss: 0.794
[3,  6000] loss: 0.818
[3,  8000] loss: 0.774
[3, 10000] loss: 0.793
[3, 12000] loss: 0.775
[4,  2000] loss: 0.687
[4,  4000] loss: 0.719
[4,  6000] loss: 0.743
[4,  8000] loss: 0.729
[4, 10000] loss: 0.737
[4, 12000] loss: 0.752
[5,  2000] loss: 0.661
[5,  4000] loss: 0.678
[5,  6000] loss: 0.693
[5,  8000] loss: 0.696
[5, 10000] loss: 0.698
[5, 12000] loss: 0.711
[6,  2000] loss: 0.633
[6,  4000] loss: 0.676
[6,  6000] loss: 0.636
[6,  8000] loss: 0.658
[6, 10000] loss: 0.666
[6, 12000] loss: 0.708
[7,  2000] loss: 0.638
[7,  4000] loss: 0.630
[7,  6000] loss: 0.661
[7,  8000] loss: 0.665
[7, 10000] loss: 0.680
[7, 12000] loss: 0.685
[8,  2000] loss: 0.606
[8, 